# 12.03 - Casos Reales de Scraping

**Autor:** Miguel Angel Vazquez Varela  
**Nivel:** Intermedio  
**Tiempo estimado:** 35 min

---

## ¿Qué aprenderás?

- Scrapear múltiples páginas (paginación)
- Manejar estructuras HTML irregulares
- Limpiar datos extraídos con pandas
- Guardar los resultados en CSV/JSON
- Implementar retries y delays responsables

---

## 1. Paginación

La mayoría de sitios dividen los resultados en páginas. Hay que detectar el patrón de la URL y iterar.

In [ ]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup


def scrape_page(url: str, headers: dict) -> BeautifulSoup | None:
    """Descarga una página y retorna el soup. Maneja errores."""
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        return BeautifulSoup(response.text, 'lxml')
    except requests.exceptions.RequestException as e:
        print(f"Error en {url}: {e}")
        return None


def scrape_paginated(base_url: str, max_pages: int = 5) -> list[dict]:
    """
    Scraping con paginación genérica.

    Parameters
    ----------
    base_url : str
        URL base con placeholder {page} para el número de página
    max_pages : int
        Número máximo de páginas a scrapear

    Returns
    -------
    list[dict]
        Lista de registros extraídos
    """
    headers = {'User-Agent': 'DataPortfolio/1.0 (educational)'}
    all_records = []

    for page in range(1, max_pages + 1):
        url = base_url.format(page=page)
        print(f"Scrapeando página {page}: {url}")

        soup = scrape_page(url, headers)
        if soup is None:
            break

        # Extraer registros de esta página (ajustar selectores al sitio real)
        rows = soup.select('table.data tr')
        if not rows:
            print(f"Sin datos en página {page}, deteniendo")
            break

        for row in rows[1:]:  # Saltar la cabecera
            cells = row.find_all('td')
            if cells:
                all_records.append({
                    'col1': cells[0].text.strip(),
                    'col2': cells[1].text.strip() if len(cells) > 1 else None,
                })

        time.sleep(1)  # Respetar el servidor: 1 segundo entre peticiones

    return all_records


print("Función de paginación definida")

---

## 2. Caso real: datos de la API de OpenData Madrid

In [ ]:
# API pública del Ayuntamiento de Madrid - datos de BiciMAD
# No requiere autenticación para datos básicos

def fetch_bicimad_data() -> pd.DataFrame | None:
    """Obtiene datos actuales de estaciones BiciMAD."""
    url = "https://api.citybik.es/v2/networks/bicimad"
    headers = {'User-Agent': 'DataPortfolio/1.0 (educational)'}

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()

        stations = data['network']['stations']
        df = pd.DataFrame(stations)

        # Limpiar y seleccionar columnas relevantes
        df = df[['name', 'free_bikes', 'empty_slots', 'latitude', 'longitude', 'timestamp']]
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df['total_docks'] = df['free_bikes'] + df['empty_slots']
        df['occupancy_pct'] = (df['free_bikes'] / df['total_docks'] * 100).round(1)

        return df

    except (requests.exceptions.RequestException, KeyError) as e:
        print(f"Error: {e}")
        return None


df_bicimad = fetch_bicimad_data()

if df_bicimad is not None:
    print(f"Estaciones: {len(df_bicimad)}")
    print(f"\nTop 5 estaciones con más bicis disponibles:")
    print(df_bicimad.nlargest(5, 'free_bikes')[['name', 'free_bikes', 'occupancy_pct']])

---

## 3. Limpieza de datos scrapeados

In [ ]:
# Los datos scrapeados suelen venir sucios: espacios, caracteres raros, tipos incorrectos

raw_data = [
    {'name': '  Sol   ', 'bikes': '12 bicis', 'lat': '40.4168°N'},
    {'name': 'Atocha\n', 'bikes': '5',       'lat': '40.4089°N'},
    {'name': 'Cibeles', 'bikes': 'N/A',      'lat': '40.4194°N'},
    {'name': 'Retiro',  'bikes': '18 bicis', 'lat': '40.4153°N'},
]

df_raw = pd.DataFrame(raw_data)
print("Datos crudos:")
print(df_raw)
print()

def clean_scraped_df(df: pd.DataFrame) -> pd.DataFrame:
    """Limpia un DataFrame con datos scrapeados típicos."""
    df = df.copy()

    # Limpiar strings
    df['name'] = df['name'].str.strip()

    # Extraer números de strings mixtos
    df['bikes'] = pd.to_numeric(
        df['bikes'].str.extract(r'(\d+)')[0],
        errors='coerce'
    )

    # Limpiar coordenadas
    df['lat'] = df['lat'].str.replace('°N', '').astype(float)

    return df


df_clean = clean_scraped_df(df_raw)
print("Datos limpios:")
print(df_clean)
print(f"\nNulos en bikes: {df_clean['bikes'].isna().sum()}")

---

## 4. Guardar los datos

In [ ]:
from pathlib import Path
from datetime import datetime

def save_scraped_data(df: pd.DataFrame, name: str, output_dir: str = 'data/raw') -> None:
    """
    Guarda datos scrapeados con timestamp en el nombre.

    Parameters
    ----------
    df : pd.DataFrame
        Datos a guardar
    name : str
        Nombre base del archivo
    output_dir : str
        Directorio de salida
    """
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

    csv_path = f"{output_dir}/{name}_{timestamp}.csv"
    df.to_csv(csv_path, index=False)
    print(f"Guardado: {csv_path} ({len(df)} filas)")


# Guardar los datos de BiciMAD si los obtuvimos
if df_bicimad is not None:
    save_scraped_data(df_bicimad, 'bicimad_stations')

---

## 5. Retries con backoff exponencial

In [ ]:
import time

def fetch_with_retry(url: str, max_retries: int = 3, backoff: float = 2.0) -> requests.Response | None:
    """
    Realiza una petición GET con reintentos y backoff exponencial.

    Parameters
    ----------
    url : str
        URL a descargar
    max_retries : int
        Número máximo de reintentos
    backoff : float
        Factor de espera (se multiplica en cada intento)
    """
    headers = {'User-Agent': 'DataPortfolio/1.0 (educational)'}
    wait = 1.0

    for attempt in range(1, max_retries + 1):
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            return response
        except requests.exceptions.RequestException as e:
            print(f"Intento {attempt}/{max_retries} fallido: {e}")
            if attempt < max_retries:
                print(f"Esperando {wait:.1f}s antes de reintentar...")
                time.sleep(wait)
                wait *= backoff  # 1s → 2s → 4s

    print(f"Fallaron todos los intentos para: {url}")
    return None


# Probar con URL válida
resp = fetch_with_retry("https://api.citybik.es/v2/networks/bicimad")
if resp:
    print(f"Éxito: {resp.status_code}")

---

## Resumen

| Técnica | Cuándo usarla |
|---|---|
| Paginación con `{page}` | URL cambia con número de página |
| `str.extract(r'(\d+)')` | Extraer números de texto mixto |
| `pd.to_numeric(..., errors='coerce')` | Convertir sin fallar en N/A |
| Timestamp en nombre de archivo | Datos que cambian con el tiempo |
| Backoff exponencial | Evitar sobrecargar el servidor |
| `time.sleep(1)` | Respetar rate limits |

---

## Ejercicio

Construye un scraper que cada 5 minutos obtenga los datos de BiciMAD, los limpie y los guarde en CSV con timestamp. Analiza cómo cambia la disponibilidad de bicis a lo largo del tiempo.

In [ ]:
# Tu solución aquí

---

**Anterior:** [12.02 - BeautifulSoup](./12_02_beautifulsoup.ipynb)  
**Siguiente:** [12.04 - Selenium](./12_04_selenium.ipynb)